In [ ]:
%matplotlib widget

In [ ]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import json

from matplotlib import  pyplot as plt
import seaborn as sns
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

In [ ]:
plt.close("all")
master =  Path(r"Z:\Hagar\old data e0040_v13 with heading")
fish_list = list(master.glob("*_f*"))
path = fish_list[0]


In [ ]:
selected = fl.load(path / "selected.h5")
rel_rois = fl.load(path / "reliable_rois.h5")['reliability_arr']

In [ ]:
in_ahb = fl.load(path / "mask_and_idxs_ahb.h5")['cells_idx']

In [ ]:
rel_hdns = rel_rois[selected]
rel_rois = rel_rois[in_ahb]
rand_rois = np.random.rand(np.shape(rel_rois)[0])
rand_hdns = np.random.rand(np.shape(rel_hdns)[0])

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(5,4), sharex=True)
ax[0].hist(rel_rois, bins=50, color='skyblue', label='All ROIs')
ax[0].hist(rel_hdns, bins=50, color='forestgreen', label='HDNs')
ax[0].legend(prop={'size': 10})

ax[1].scatter(rel_rois, rand_rois, color='skyblue', s=5)
ax[1].scatter(rel_hdns, rand_hdns, color='forestgreen', s=5)

ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)

ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)

ax[1].set_xlabel('Reliability of tuning to visual motion')
ax[0].set_ylabel('# Neurons')
plt.subplots_adjust(hspace=0.4, bottom=0.2)

In [ ]:
fig.savefig(path / "hdns vs rel index in ahb.jpg", dpi=300)

In [ ]:
all_data = [rel_hdns, rel_rois]
fig1, ax1 = plt.subplots(1,1, figsize=(3,3), sharex=True)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.violinplot(all_data,
                  showmeans=False,
                  showmedians=True)
ax1.set_xticklabels([' ', 'HDNs', ' ', 'All ROIs'])


In [ ]:
fig1.savefig(path / "hdns vs rel index violin.jpg", dpi=300)

In [ ]:
from lotr import LotrExperiment

In [ ]:
traces = fl.load(path / "filtered_traces.h5", "/detr")
exp = LotrExperiment(path)

In [ ]:
t_start_s = 50
t_end = exp.n_pts//2
t_lims = (t_start_s*exp.fn, t_end)
t_slice = slice(*t_lims)
cc_wnd = 3600
i_array = np.arange(t_slice.start, t_slice.stop, cc_wnd*exp.fn)
cc_mats = np.zeros((traces.shape[1], traces.shape[1], len(i_array)))

for n, i in enumerate(i_array):
    cc_mats[:, :, n] = np.corrcoef(traces[i:i + cc_wnd*exp.fn, :].T)
corr_mat = np.nanmean(cc_mats, 2)

In [ ]:
neg_corr = np.nanmin(corr_mat, axis=0)

neg_corr_hdns = neg_corr[selected]
neg_corr_ahb = neg_corr[in_ahb]

In [ ]:
selected_vis = np.where(rel_rois > 0.3)[0]
rel_vis = rel_rois[selected_vis]
neg_corr_vis =  neg_corr_ahb[selected_vis]

In [ ]:
fig2, ax2 = plt.subplots(1, 2, figsize=(10,5), sharex=True)

ax2[0].hist(rel_rois, bins=50, color='skyblue', label='All ROIs')
ax2[0].hist(rel_hdns, bins=50, color='forestgreen', label='HDNs')
ax2[0].set_ylabel('# Neurons')
ax2[0].legend(prop={'size': 10})

ax2[0].spines['top'].set_visible(False)
ax2[0].spines['right'].set_visible(False)

ax2[1].scatter(rel_rois, neg_corr_ahb, color='skyblue', s=5, label='All ROIs')
ax2[1].scatter(rel_vis, neg_corr_vis, color='purple', s=5, label='Visually Tuned')
ax2[1].scatter(rel_hdns, neg_corr_hdns, color='forestgreen', s=5, label='HDNs')

ax2[1].legend(prop={'size': 10})

ax2[1].spines['top'].set_visible(False)
ax2[1].spines['right'].set_visible(False)

ax2[1].set_xlabel('Reliability of tuning to visual motion')
ax2[0].set_xlabel('Reliability of tuning to visual motion')
ax2[1].set_ylabel('Max negative correlation')
ax2[1].set_xlim([-0.2, 1])
ax2[0].set_xlim([-0.2, 1])
plt.subplots_adjust(left=0.1, bottom=0.2)

In [ ]:
fig2.savefig(path / "hdns vs rel index scatter2d with all neurons.jpg", dpi=300)
fig2.savefig(path / "hdns vs rel index scatter2d with all neurons.pdf", dpi=300)

In [ ]:
fig3, ax3 = plt.subplots(1, 2, figsize=(7,4), sharex=True)

ax3[0].hist(rel_rois, bins=50, color='skyblue', label='All ROIs')
ax3[0].hist(rel_hdns, bins=50, color='forestgreen', label='HDNs')
ax3[0].set_ylabel('# Neurons')
ax3[0].legend(prop={'size': 10})

ax3[0].spines['top'].set_visible(False)
ax3[0].spines['right'].set_visible(False)

ax3[1].scatter(rel_vis, neg_corr_vis, color='purple', s=5, label='Visually Tuned')
ax3[1].scatter(rel_hdns, neg_corr_hdns, color='forestgreen', s=5, label='HDNs')

ax3[1].legend(prop={'size': 10})
ax3[1].set_ylim([-1, 0])
ax3[1].set_xlim([-0.3, 1])
ax3[0].set_xlim([-0.3, 1])

ax3[1].spines['top'].set_visible(False)
ax3[1].spines['right'].set_visible(False)

ax3[1].set_xlabel('Reliability of tuning to visual motion')
ax3[0].set_xlabel('Reliability of tuning to visual motion')
ax3[1].set_ylabel('Max negative correlation')
plt.subplots_adjust(left=0.1, bottom=0.2, wspace=0.3, right=0.95)

In [ ]:
fig3.savefig(path / "hdns vs rel index scatter2d.jpg", dpi=300)
fig3.savefig(path / "hdns vs rel index scatter2d.pdf", dpi=300)